In [ ]:
import sys
from glob import glob
import xarray as xr
import os.path as path 
import matplotlib.pyplot as plt 

from dask.distributed import Client

sys.path.append('../../src/thermal') 

from open import dataset as open_dataset, mf_dataset as open_mfdataset


def strip_Temp(fp): 
    return float(fp.split('Tma_')[-1].split('_')[0])

def preproccess(ds): 
    # get filepath from datasource
    fp = ds.encoding["source"]
    
    # find ensemble parameter values
    T_ma = strip_Temp(fp)
    
    # expand the dimmension to allow concatetnation along 
    ds = ds.expand_dims("T_ma").assign_coords(T_ma=('T_ma', [T_ma]))
    
    # Thin the data by a factor of ten to speed things up. 
    ds = ds.isel(t=slice(0,-1,10))
    
    return ds

def calc_perc_temp(src): 
    # number of valid nodes above minimum ice-thickness
    # NN = src.height.where(src.height < 10).count(dim=('nMesh_node'))#.count(dim=('coord_1', 'coord_2'))
    

    NN = src.height.where(src.height > 10).count(dim=('coord_1', 'coord_2'))

    # mask for determining where nodes are temperate 
    mask = (src['enthalpy_h'] > src['phase change enthalpy']) & (src.height > 10)
    # number of nodes where ethalpy exceeds phase chnage enthalpy
    NT = src.enthalpy_h.where(mask).count(dim=('coord_1', 'coord_2'))
    return NT/NN * 100

In [ ]:
client = Client()

In [ ]:
%%time 

src_fp = '/Volumes/thermal_ssd/DataDir/glc1-a_dx_50_NT_2000_dt_1.0_MB_*_OFF_Tma_-7.*_prog_gridded.nc'
test   = xr.open_mfdataset(src_fp, parallel=True)

CPU times: user 2.09 s, sys: 1.81 s, total: 3.9 s
Wall time: 8.45 s


In [ ]:
test

0.010280168

In [ ]:
subset = test.isel(t=-1, T_ma=slice(0,None,2))

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(10, 2), 
                         sharex=True, sharey=True)

for i, ax in enumerate(axes[0,:].ravel()):
    
    ax.pcolormesh(subset.X[:,::-1], 
                  subset.Z.isel(T_ma=i, Delta_MB=0), 
                  subset['temperature'].isel(T_ma=i, Delta_MB=0), 
                   shading='gouraud')
    

for i, ax in enumerate(axes[1,:].ravel()):
    
    ax.pcolormesh(subset.X[:,::-1], 
                  subset.Z.isel(T_ma=i, Delta_MB=-1), 
                  subset['temperature'].isel(T_ma=i, Delta_MB=-1), 
                   shading='gouraud')
    
ax.set_xlim(None, 6e3)
ax.set_ylim(1.75e3, None)

In [ ]:
subset['temperature'].isel(T_ma=i)

In [ ]:
Vol = test.height.isel(coord_2=-1).integrate("coord_1") /\
      test.height.isel(coord_2=-1).isel(t=0).integrate("coord_1")

In [ ]:
Volume = Vol.compute()

In [ ]:
Volume.sel(t=200)

In [ ]:
# src_dir = '/Volumes/thermal/Thesis/thermal-structure/study/coupled_init/'
# src_fp  = path.join(src_dir, 'glc1-a/nc/glc1-a_dx_50_NT_2000_dt_1.0_MB_-1.500_OFF_Tma_-7.50_prog.nc')

In [ ]:
%%time 

src_fp  = '/Volumes/thermal_ssd/DataDir/glc1-a_dx_50_NT_2000_dt_1.0_MB_-1.075_OFF_Tma_*_prog.nc'

test = open_mfdataset(src_fp, concat_dim='T_ma', preprocess=preproccess, parallel=True)
test

In [ ]:
Vol = test.height.isel(coord_2=-1).integrate("coord_1") /\
      test.height.isel(coord_2=-1).isel(t=0).integrate("coord_1")

In [ ]:
Vol.plot(hue='T_ma');

In [ ]:
calc_perc_temp(test).plot(hue='T_ma');

In [ ]:
%%time

# load the external hard drive
ds = open_dataset(src_fp)

In [ ]:
%%time

src_fp = '/Users/andrewnolan/Desktop/glc1-a_dx_50_NT_2000_dt_1.0_MB_-1.500_OFF_Tma_-7.50_prog.nc'

# load directly from my laptops SSD
ds = open_dataset(src_fp)

In [ ]:
with xr.open_dataset(src_fp) as src: 
    pass

In [ ]:
src.attrs

In [ ]:
ds

In [ ]:
%%time

src_fp = '/Volumes/Extreme_SSD/glc1-a_dx_50_NT_2000_dt_1.0_MB_-1.500_OFF_Tma_-7.50_prog.nc'

# load directly from Emily's SSD
ds = open_dataset(src_fp)

In [ ]:
%%time 

src_fp = '/Volumes/Glacier/glc1-a_dx_50_NT_2000_dt_1.0_MB_-1.500_OFF_Tma_-7.50_prog.nc'

ds = open_dataset(src_fp)

In [ ]:
test = open_mfdataset(src_fp, preprocess=preproccess)
test

In [ ]:
Vol = ds.height.isel(coord_2=-1).integrate("coord_1") /\
      ds.height.isel(coord_2=-1).isel(t=0).integrate("coord_1")

def calc_perc_temp(src): 
    # number of valid nodes above minimum ice-thickness
    # NN = src.height.where(src.height < 10).count(dim=('nMesh_node'))#.count(dim=('coord_1', 'coord_2'))
    

    NN = src.height.where(src.height > 10).count(dim=('coord_1', 'coord_2'))

    # mask for determining where nodes are temperate 
    mask = (src['enthalpy_h'] > src['phase change enthalpy']) & (src.height > 10)
    # number of nodes where ethalpy exceeds phase chnage enthalpy
    NT = src.enthalpy_h.where(mask).count(dim=('coord_1', 'coord_2'))
    return NT/NN * 100

In [ ]:
Vol.plot()

In [ ]:
calc_perc_temp(ds).plot()

In [ ]:
fig, ax = plt.subplots()

im = ax.pcolormesh(ds.X[:,::-1], 
                   ds.Z.isel(t=-3), 
                   ds['temperature'].isel(t=-1), 
                   shading='gouraud')

fig.colorbar(im)

In [ ]:
files = 'fdsjlkd'

(type(files) == str) and ('*' not in files)